# Freshwater discharge at depth in southeast Greenland fjords

## Purpose
This notebook generates figures and spreadsheets of freshwater discharge as a function of depth for each fjord in the study, and cumulatively across all fjords. Throughout this notebook, 'discharge' refers to freshwater discharge only (no solid discharge).

The figures show, for each fjord, the discharge above/below a threshold depth, and binned by depth across the whole depth range, for discharge originating from either land or ice basins. There are also depth-binned figures that are cumulative across all fjords. 

The spreadsheets detail the discharge as a function of depth for each fjord, and the mean annual discharge for each fjord. 

## Requirements
- `../freshwater/ice/{MAR, RACMO}.nc`
- `../freshwater/land/{MAR, RACMO}.nc`
- `./fjord_outlets`

If you are generating your own data rather than reproducing our analysis, see instructions in README.

## User parameters
- data model source (MAR or RACMO)
- list of fjord ID numbers
- threshold depth for calculating discharge above/below threshold
- altitude intervals for depth binning
- start and end dates
- colormap details

## Output
- Figures of freshwater discharge above/below a threshold depth in each fjord \
`../figures/fjord_FWdischarge_depththreshold_{MAR, RACMO}/fjord*_FWdischarge_threshold_{MAR, RACMO}.png`
- Figures of freshwater discharge binned by depth across the whole depth range \
`../figures/fjord_FWdischarge_depthbinned_{MAR, RACMO}/fjord*_FWdischarge_depthbinned_{MAR, RACMO}.png`
- Figure of freshwater discharge time series binned by depth and combined across all fjords, separated by land-sourced and ice-sourced discharge \
`../figures/allfjords_FWdischarge_depthbinned_{MAR, RACMO}_landvsice.png`
- Figure of freshwater discharge time series binned by depth and combined across all fjords, land and ice sources summed together \
`../figures/allfjords_FWdischarge_depthbinned_{MAR, RACMO}_combined.png`
- Spreadsheets of freshwater discharge time series for individual fjords, binned by depth and summed across land and ice sources \
`../databases/fjord_FWdischarge_depthbinned_{MAR, RACMO}_combined/fjord*_FWdischarge_depthbinned_{MAR, RACMO}.csv`
- Spreadsheet of mean annual deep ice+land freshwater discharge for each fjord \
`../databases/fjord_FWdischarge_mean_annual_deep_{MAR, RACMO}.csv`

## Code

In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.cm import ScalarMappable
import numpy as np
import copy
import pathlib

In [ ]:
# USER PARAMETERS

# -- Choose to load ice and land discharge from either RACMO or MAR
source = 'MAR' # choose 'RACMO' or 'MAR'

# -- Get list of fjord numbers
fjord_number_list = list(range(1,53))

# -- Set threshold depth for above/below discharge calculations (units=m)
threshold_depth = -200

# -- Define altitude intervals for depth binning (units=m)
interval = 20
top = 20
shallow_bins = np.arange(threshold_depth, top+interval, interval)
all_bins = np.arange(-1000, top+interval, interval)
alt_bins = pd.interval_range(start=-1000, end=top, freq=interval)

# -- Get list of dates for timespan of data (format='YYYY-MM-DD')
time_start = pd.to_datetime('2015-01-01')
time_end = pd.to_datetime('2019-12-31')
time = pd.date_range(start=time_start, end=time_end)

# -- Define Blues cmap with nodata color as maroon, under_value color as silver
cmap = copy.copy(plt.cm.get_cmap('Blues'))
cmap.set_bad(color='dimgray')
cmap.set_under(color='silver')

In [ ]:
# Load data
discharge_ice = xr.open_dataset(f'../freshwater/ice/{source}.nc', engine='h5netcdf')
discharge_land = xr.open_dataset(f'../freshwater/land/{source}.nc', engine='h5netcdf')

In [ ]:
def roundup(val, order=None):
    '''Round value up to next given order of magnitude, used for setting colorbar limits'''
    if order is None:
        order = np.floor(np.log10(val))
    val_roundup = np.ceil(val / 10**(order)) * 10**(order)
    return val_roundup

def discharge_from_outlets(oi, ol):
    # MODIFIED FROM KEN MANKOFF
    ## oi & ol are outlets for ice and land respectively
    
    ## simple method, but fails if ol contains stations not in discharge_land
    ## NOTE: not sure how that occurs
    ## See https://github.com/pydata/xarray/issues/1990
    # ri = discharge_ice.sel(station = oi)
    # rl = discharge_land.sel(station = ol, method='nearest')

    # update 2023-06-07: breaks if no stations present. Added if/else.
    # If ice/land outlets are present, get discharge at them and sum
    if not oi.empty:
        ri = discharge_ice.reindex({'station':oi.id.values})
        ri = ri.discharge.sum(dim='station', skipna=True).to_dataframe()
    else:
        ri = xr.full_like(discharge_ice.sel(station=slice(1,2)), fill_value=np.nan)
        ri = ri.discharge.sum(dim='station', skipna=False).to_dataframe()
    if not ol.empty:
        rl = discharge_land.reindex({'station':ol.id.values})
        rl = rl.discharge.sum(dim='station', skipna=True).to_dataframe()
    else:
        rl = xr.full_like(discharge_land.sel(station=slice(1,2)), fill_value=np.nan)
        rl = rl.discharge.sum(dim='station', skipna=False).to_dataframe()

    return ri, rl


def discharge_from_outlets_binned(oi, ol, bins):
    # bins can be an integer or a list (e.g. np.arange(-200, 250, 50))

    # update 2023-06-07: breaks if no stations present. Added if/else.
    # If ice/land outlets are present, get discharge at them
    if not oi.empty:
        ri = discharge_ice.reindex({'station':oi.id.values})
    else:
        ri = xr.full_like(discharge_ice.sel(station=slice(1,2)), fill_value=np.nan)
        ri['alt'] = np.nan # force nan altitude for fake dataframe so binning works
    if not ol.empty:
        rl = discharge_land.reindex({'station':ol.id.values})
    else:
        rl = xr.full_like(discharge_land.sel(station=slice(1,2)), fill_value=np.nan)
        rl['alt'] = np.nan # force nan altitude for fake dataframe so binning works

    # bin discharge by altitude (depth)
    if ri.alt.values.size == 0 or np.isnan(ri.alt.values).all() or (type(bins) is not int and (ri.alt.values > bins.max()).all()):
        ri_bin = None
    else: 
        ri.alt.values = np.clip(ri.alt.values, a_min=None, a_max=1) # clip all values above 0 so that they're in top bin, regardless of bin edge
        ri_bin = ri.groupby_bins(group=ri.alt, bins=bins).sum()
    if rl.alt.values.size == 0 or np.isnan(rl.alt.values).all() or (type(bins) is not int and (ri.alt.values > bins.max()).all()):
        rl_bin = None
    else: 
        rl.alt.values = np.clip(rl.alt.values, a_min=None, a_max=1) # clip all values above 0 so that they're in top bin, regardless of bin edge
        rl_bin = rl.groupby_bins(group=rl.alt, bins=bins).sum()

    return ri_bin, rl_bin

In [ ]:
# Example discharge-at-depth plot for a single fjord
fjord_number = 13
outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')
# Separate land and ice outlets
o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])
# Separate deep and shallow outlets (relative to a threshold depth)
o_land_deep = o_land.where(o_land.elev <= threshold_depth).dropna(subset=['elev'])
o_land_shal = o_land.where(o_land.elev > threshold_depth).dropna(subset=['elev'])
o_ice_deep = o_ice.where(o_ice.elev <= threshold_depth).dropna(subset=['elev'])
o_ice_shal = o_ice.where(o_ice.elev > threshold_depth).dropna(subset=['elev'])

ris_bin, rls_bin = discharge_from_outlets_binned(o_ice_shal, o_land_shal, bins=shallow_bins)

fig, ax = plt.subplots()
ris_bin.discharge.plot(
    ax=ax, 
    cmap=cmap, 
    norm=colors.LogNorm(vmin=1, vmax=roundup(ris_bin.discharge.max().values), clip=False),
    cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'}
)
start, end = ax.get_ylim()
ax.yaxis.set_ticks(np.arange(start, end, interval))
ax.set_ylabel('Altitude (m asl)')
ax.set_xlabel('Time')
plt.show()

In [ ]:
# Generate discharge plots for each fjord

# -- Loop through each fjord and save plots of discharge above/below a threshold depth
for fjord_number in fjord_number_list:
    print(f'Processing Fjord {fjord_number}...')
    outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')

    # Separate land and ice outlets
    o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
    o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])

    # Separate deep and shallow outlets (relative to a threshold depth)
    o_land_deep = o_land.where(o_land.elev <= threshold_depth).dropna(subset=['elev'])
    o_land_shal = o_land.where(o_land.elev > threshold_depth).dropna(subset=['elev'])
    o_ice_deep = o_ice.where(o_ice.elev <= threshold_depth).dropna(subset=['elev'])
    o_ice_shal = o_ice.where(o_ice.elev > threshold_depth).dropna(subset=['elev'])

    # THRESHOLD
    # Get discharge from shallow/deep land/ice outlets
    ris, rls = discharge_from_outlets(o_ice_shal, o_land_shal)
    rid, rld = discharge_from_outlets(o_ice_deep, o_land_deep)
    # Plot discharge above/below threshold depth for land and ice outlets
    fig, axs = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(8,4), layout='constrained')
    axs = axs.flatten()
    fig.suptitle(f'Fjord {fjord_number}')
    # -- Plot shallow discharge
    axs[0].plot(ris, label='ice basins')
    axs[0].plot(rls, label='land basins')
    axs[0].legend(loc='upper left', bbox_to_anchor=(0.01, 0.99))
    axs[0].set_title(f'Shallow FW discharge (above {threshold_depth}m)')
    axs[0].set_ylabel('Discharge ($\mathrm{m^3 s^{-1}}$)')
    axs[0].set_xlabel('Time')
    axs[0].grid(True)
    if (ris.discharge.isnull().all() & rls.discharge.isnull().all()):
        axs[0].text(
            x=0.5,
            y=0.5,
            s='NO DATA',
            fontsize=20,
            horizontalalignment='center',
            verticalalignment='center',
            transform=axs[0].transAxes
        )
        axs[0].set_yticks([])
        axs[0].grid(False)
    # -- Plot deep discharge
    axs[1].plot(rid, label='ice basins')
    axs[1].plot(rld, label='land basins')
    axs[1].set_title(f'Deep FW discharge (below {threshold_depth}m)')
    axs[1].set_xlabel('Time')
    axs[1].grid(True)
    if (rid.discharge.isnull().all() & rld.discharge.isnull().all()):
        axs[1].text(
            x=0.5,
            y=0.5,
            s='NO DATA',
            fontsize=20,
            horizontalalignment='center',
            verticalalignment='center',
            transform=axs[1].transAxes
        )
        axs[1].set_yticks([])
        axs[1].grid(False)
    # -- Save output
    pathlib.Path(f'../figures/fjord_FWdischarge_depththreshold_{source}/').mkdir(parents=True, exist_ok=True)
    plt.savefig(
        f'../figures/fjord_FWdischarge_depththreshold_{source}/fjord{fjord_number:02}_FWdischarge_threshold_{source}.png',
        bbox_inches='tight',
        dpi=300,
        facecolor='white'
    )
    plt.close()

    # BINNING
    # Get discharge binned at depths for ice and land shallow and deep outlets
    ris_bin, rls_bin = discharge_from_outlets_binned(o_ice_shal, o_land_shal, bins=shallow_bins)
    rid_bin, rld_bin = discharge_from_outlets_binned(o_ice_deep, o_land_deep, bins=10)
    max_discharge = max([da.discharge.max() for da in [ris_bin, rls_bin, rid_bin, rld_bin] if da is not None]).values
    
    # Plot discharge time series binned by depth
    fig, axs = plt.subplots(nrows=2, ncols=2, sharex=True, figsize=(8,7), layout='constrained')
    axs = axs.flatten()
    fig.suptitle(f'Fjord {fjord_number}')

    # -- Plot shallow ice binned discharge
    if ris_bin is not None:
        ris_bin.discharge.plot(
            ax=axs[0], 
            cmap=cmap,
            norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False),
            cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'}
        )
        start, end = axs[0].get_ylim()
        axs[0].yaxis.set_ticks(np.arange(start, end, interval))
    else:
        axs[0].text(
            x=0.5,
            y=0.5,
            s='NO DATA',
            fontsize=20,
            horizontalalignment='center',
            verticalalignment='center',
            transform=axs[0].transAxes
        )
        fig.colorbar(
            ScalarMappable(
                cmap=cmap,
                norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False)
            ),
            ax=axs[0], 
            extend='min', 
            label='Discharge ($\mathrm{m^3 s^{-1}}$)'
        )
        axs[0].set_yticks([])
    axs[0].set_title('Shallow ice FW discharge')
    axs[0].set_ylabel('Altitude (m asl)')
    axs[0].xaxis.label.set_visible(False)
    
    # -- Plot shallow land binned discharge
    if rls_bin is not None:
        rls_bin.discharge.plot(
            ax=axs[1],
            cmap=cmap,
            norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False),
            cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'}
        )
        start, end = axs[1].get_ylim()
        axs[1].yaxis.set_ticks(np.arange(start, end, interval))
    else:
        axs[1].text(
            x=0.5,
            y=0.5,
            s='NO DATA',
            fontsize=20,
            horizontalalignment='center',
            verticalalignment='center',
            transform=axs[1].transAxes
        )
        fig.colorbar(
            ScalarMappable(
                cmap=cmap,
                norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False)
            ),
            ax=axs[1],
            extend='min',
            label='Discharge ($\mathrm{m^3 s^{-1}}$)'
        )
        axs[1].set_yticks([])
    axs[1].set_title('Shallow land FW discharge')
    axs[1].yaxis.label.set_visible(False)
    axs[1].xaxis.label.set_visible(False)

    # -- Plot deep ice binned discharge
    if rid_bin is not None:
        rid_bin.discharge.plot(
            ax=axs[2],
            cmap=cmap,
            norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False),
            cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'})
    else:
        axs[2].text(
            x=0.5,
            y=0.5,
            s='NO DATA',
            fontsize=20,
            horizontalalignment='center',
            verticalalignment='center',
            transform=axs[2].transAxes
        )
        fig.colorbar(
            ScalarMappable(
                cmap=cmap,
                norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False)
            ),
            ax=axs[2],
            extend='min',
            label='Discharge ($\mathrm{m^3 s^{-1}}$)'
        )
        axs[2].set_yticks([])
    axs[2].set_title('Deep ice FW discharge')
    axs[2].set_ylabel('Altitude (m asl)')
    axs[2].set_xlabel('Time')

    # -- Plot deep land binned discharge
    if rld_bin is not None:
        rld_bin.discharge.plot(
            ax=axs[3],
            cmap=cmap,
            norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False),
            cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'}
        )
    else:
        axs[3].text(
            x=0.5,
            y=0.5,
            s='NO DATA',
            fontsize=20,
            horizontalalignment='center', 
            verticalalignment='center', 
            transform=axs[3].transAxes
        )
        fig.colorbar(
            ScalarMappable(
                cmap=cmap, 
                norm=colors.LogNorm(vmin=1, vmax=roundup(max_discharge), clip=False)
            ),
            ax=axs[3],
            extend='min',
            label='Discharge ($\mathrm{m^3 s^{-1}}$)'
            )
        axs[3].set_yticks([])
    axs[3].set_title('Deep land FW discharge')
    axs[3].yaxis.label.set_visible(False)
    axs[3].set_xlabel('Time')
    
    # -- Save output
    pathlib.Path(f'../figures/fjord_FWdischarge_depthbinned_{source}/').mkdir(parents=True, exist_ok=True)
    plt.savefig(
        f'../figures/fjord_FWdischarge_depthbinned_{source}/fjord{fjord_number:02}_FWdischarge_depthbinned_{source}.png',
        bbox_inches='tight', 
        dpi=300, 
        facecolor='white'
    )
    plt.close()

In [ ]:
# Store ice and land discharge data in depth bins for each fjord

# -- Initialize dataarrays for storage
fjorddischarge_ice = xr.DataArray(
    dims=['alt_bins', 'time', 'fjord'],
    coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number_list)
)
fjorddischarge_land = xr.DataArray(
    dims=['alt_bins', 'time', 'fjord'],
    coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number_list)
)

# -- Loop through each fjord and save plots of discharge vs depth
for fjord_number in fjord_number_list:
    print(f'Processing Fjord {fjord_number}...')
    outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')

    # Separate land and ice outlets
    o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
    o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])

    # Get discharge from shallow/deep land/ice outlets
    ri, rl = discharge_from_outlets_binned(o_ice, o_land, bins=all_bins)
    
    # Add discharge data to dataarrays
    if ri is not None:
        ri_da = xr.DataArray(ri.discharge, coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number))
        fjorddischarge_ice.loc[dict(fjord=fjord_number)] = ri_da
    if rl is not None:
        rl_da = xr.DataArray(rl.discharge, coords=dict(alt_bins=alt_bins, time=time, fjord=fjord_number))
        fjorddischarge_land.loc[dict(fjord=fjord_number)] = rl_da

In [ ]:
# Plot land and ice basin discharge time series binned by depth, for all fjords combined
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), sharex=True, sharey=True, layout='constrained')
axs = axs.flatten()
max_sumdischarge = max([da.sum(dim='fjord').max().values for da in [fjorddischarge_ice, fjorddischarge_land] if da is not None])
ceil_max_sumdischarge = roundup(max_sumdischarge.max(), order=4)

# -- Plot binned ice discharge
fjorddischarge_ice.sum(dim='fjord').plot(
    ax=axs[0], 
    cmap=cmap,
    norm=colors.LogNorm(vmin=1, vmax=ceil_max_sumdischarge, clip=False),
    cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'}
)
axs[0].set_title('FW discharge originating from ice basins')
axs[0].set_ylabel('Altitude (m asl)')
axs[0].set_xlabel('Time')
start, end = axs[0].get_ylim()
axs[0].yaxis.set_ticks(np.arange(start, end, 100))
axs[0].xaxis.set_tick_params(reset=True)

# -- Plot binned land discharge
fjorddischarge_land.sum(dim='fjord').plot(
    ax=axs[1],
    cmap=cmap,
    norm=colors.LogNorm(vmin=1, vmax=ceil_max_sumdischarge, clip=False),
    cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'}
)
axs[1].set_title('FW discharge originating from land basins')
axs[1].set_xlabel('Time')
axs[1].yaxis.label.set_visible(False)
start, end = axs[1].get_ylim()
axs[1].yaxis.set_ticks(np.arange(start, end, 100))
axs[1].xaxis.set_tick_params(reset=True)

# -- Save output
pathlib.Path('../figures/').mkdir(parents=True, exist_ok=True)
plt.savefig(
    f'../figures/allfjords_FWdischarge_depthbinned_{source}_landvsice.png',
    bbox_inches='tight',
    dpi=300,
    facecolor='white'
)

In [ ]:
# Plot summed ice+land discharge time series binned by depth
# -- concat along arbitrary axis helps to prevent nan issues when summing
fjorddischarge_total = xr.concat((fjorddischarge_ice, fjorddischarge_land), 'z').sum(dim='z')
ceil_totaldischarge = roundup(fjorddischarge_total.sum(dim='fjord').max().values.max(), 4)
fig, ax = plt.subplots(figsize=(6, 4.5), layout='constrained')
fjorddischarge_total.sum(dim='fjord').plot(
    ax=ax, 
    cmap=cmap, 
    norm=colors.LogNorm(vmin=1, vmax=ceil_totaldischarge, clip=False),
    cbar_kwargs={'label': 'Discharge ($\mathrm{m^3 s^{-1}}$)'}
)
ax.set_title('Total FW discharge')
ax.set_ylabel('Altitude (m asl)')
ax.set_xlabel('Time')
start, end = ax.get_ylim()
ax.yaxis.set_ticks(np.arange(start, end, 100))
ax.xaxis.set_tick_params(reset=True)

pathlib.Path('../figures/').mkdir(parents=True, exist_ok=True)
plt.savefig(
    f'../figures/allfjords_FWdischarge_depthbinned_{source}_combined.png',
    bbox_inches='tight',
    dpi=300,
    facecolor='white'
)

In [ ]:
# Save depth-binned freshwater flux time series for each fjord to CSV (round to 3 decimal places)
pathlib.Path(f'../databases/fjord_FWdischarge_depthbinned_{source}_combined').mkdir(parents=True, exist_ok=True)
fjorddischarge_total = fjorddischarge_total.round(decimals=3)
for f in fjord_number_list:
    fjorddischarge_total.sel(fjord=f).to_pandas().transpose().to_csv(
        f'../databases/fjord_FWdischarge_depthbinned_{source}_combined/fjord{f:02}_FWdischarge_depthbinned_{source}.csv', 
        na_rep=np.nan,
        header=True,
        index=True
    )

In [ ]:
# Get mean annual deep (ice+land) discharge for each fjord and save to CSV (round to 3 decimal places)

# -- Initialize empty series to store mean annual deep discharge
deep_discharge = pd.Series(index=fjord_number_list, name='discharge', dtype='float64')

# -- Loop through each fjord and save plots of discharge vs depth
for fjord_number in fjord_number_list:
    print(f'Processing Fjord {fjord_number}...')
    outlets = pd.read_csv(f'./fjord_outlets/fjord{fjord_number:02}_outlets.txt')

    # Separate land and ice outlets
    o_land = outlets.where(outlets.domain == 'land').dropna(subset=['domain'])
    o_ice = outlets.where(outlets.domain == 'ice').dropna(subset=['domain'])

    # Separate deep and shallow outlets (relative to a threshold depth)
    o_land_deep = o_land.where(o_land.elev <= threshold_depth).dropna(subset=['elev'])
    o_ice_deep = o_ice.where(o_ice.elev <= threshold_depth).dropna(subset=['elev'])

    # Get discharge from shallow/deep land/ice outlets
    rid, rld = discharge_from_outlets(o_ice_deep, o_land_deep)

    # Get mean annual combined (ice+land) discharge at depth
    rd = rid.add(rld, fill_value=0) # convert nan to 0 in either df before adding
    rd = rd * 86400 # convert from m3/s to m3/day
    mean_annual_deep_discharge = rd.groupby(rd.index.year).sum().mean()
    deep_discharge[fjord_number] = mean_annual_deep_discharge.round(decimals=3)

pathlib.Path('../databases/').mkdir(parents=True, exist_ok=True)
deep_discharge.to_csv(
    f'../databases/fjord_FWdischarge_mean_annual_deep_{source}.csv',
    index_label='fjord',
    header=['mean annual discharge (m3)']
)